In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

In [2]:
# define tools

from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Bangaluru")
print(search_results)

# other tools after creation must be put in a list for reference
tools = [search]

{'query': 'What is the weather in Bangaluru', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Bengaluru', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755236153, 'localtime': '2025-08-15 11:05'}, 'current': {'last_updated_epoch': 1755235800, 'last_updated': '2025-08-15 11:00', 'temp_c': 24.2, 'temp_f': 75.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 19.0, 'wind_kph': 30.6, 'wind_degree': 260, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.09, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 25.7, 'feelslike_f': 78.3, 'windchill_c': 24.5, 'windchill_f': 76.1, 'heatindex_c': 26.0, 'heatindex_f': 78.7, 'dewpoint_c': 17.6, 'dewpoint_f': 63.6, 'vis_km': 

In [3]:
# using llm to call the tool
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [5]:
# mind model with tools
model_with_tools = model.bind_tools(tools)

In [6]:
query = "hi"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [7]:
query = "Search for the weather in Bengaluru"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Bengaluru'}, 'id': '941c8b79-55e0-40e0-8a68-5bd24a0b25b3', 'type': 'tool_call'}]


In [15]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools) # bind_tools will be called under the hood

In [9]:
input_message = {"role": "user", "content": "hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [10]:
input_message = {"role": "user", "content": "whats the weather in Bengaluru"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

whats the weather in Bengaluru
================================== Ai Message ==================================
Tool Calls:
  tavily_search (a76c314d-2042-4109-bb67-5f77c17fdf73)
 Call ID: a76c314d-2042-4109-bb67-5f77c17fdf73
  Args:
    query: weather in Bengaluru
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Bengaluru", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Bengaluru", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755237291, 'localtime': '2025-08-15 11:24'}, 'current': {'last_updated_epoch': 1755236700, 'last_updated': '2025-08-15 11:15', 'temp_c': 25.1, 'temp_f': 77.2, 'is_day': 1, 'condition': {'text': 'Partly clou

In [13]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in Bengaluru
================================== Ai Message ==================================
Tool Calls:
  tavily_search (90388137-56bb-4317-ae5c-c5e2b023b5ec)
 Call ID: 90388137-56bb-4317-ae5c-c5e2b023b5ec
  Args:
    query: weather in Bengaluru
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Bengaluru", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Bengaluru", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Bengaluru', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755237291, 'localtime': '2025-08-15 11:24'}, 'current': {'last_updated_epoch': 1755236700, 'last_updated': '2025-08-15 11:15', 'temp_c': 25.1, 'temp_f': 77.2, 'is_day': 1, 'condition': {'text': 'Partly clou

In [16]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in Bengaluru on 2025-08-15 11:24 is partly cloudy with a temperature of 25.1°C (77.2°F). The wind speed is |30.6 kph and the humidity is 79%.|

In [ ]:
# adding memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [30]:
agent_executor = create_react_agent(model,tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [31]:
input_message = {"role": "user", "content": "Hi, I'm Durlav!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Durlav!
================================== Ai Message ==================================

Hello Durlav! How can I help you today?


In [32]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I'm sorry, I don't have memory of past conversations. Could you please tell me your name again?
